In [1]:
import re
import csv
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [2]:
# Read Dataset
train_dataset = pd.read_csv('./Data/Training.csv')
test_dataset = pd.read_csv('./Data/Testing.csv')

In [3]:
# Data Preparation 
# Split Data fitur/gejala (x) dan label/penyakit (y)
cols_train= train_dataset.columns
cols_train= cols_train[:-1]
cols_test= test_dataset.columns
cols_test= cols_test[:-1]
x_train = train_dataset[cols_train]
y_train = train_dataset['prognosis']
x_test = test_dataset[cols_test]
y_test = test_dataset['prognosis']


In [4]:
label_encoder = LabelEncoder()
# Fit label encoder and return encoded labels.
y_train = label_encoder.fit_transform(y_train)
# Transform labels to normalized encoding.
y_test = label_encoder.transform(y_test)

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(len(cols_train),)), # Ada 132 kelas gejala, bisa juga ganti jadi len(cols_train) = 132
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(set(y_train)), activation='softmax') # Ada 41 kelas penyakit, bisa juga ganti jadi len(set(y)) = 41
])

model.compile(optimizer='adam', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])

clf = model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
154/154 [==============================] - 7s 11ms/step - loss: 1.2562 - accuracy: 0.8827
Epoch 2/10
154/154 [==============================] - 2s 12ms/step - loss: 0.0189 - accuracy: 1.0000
Epoch 3/10
154/154 [==============================] - 2s 14ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 4/10
154/154 [==============================] - 2s 14ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 5/10
154/154 [==============================] - 2s 13ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 6/10
154/154 [==============================] - 2s 10ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 7/10
154/154 [==============================] - 1s 9ms/step - loss: 8.6169e-04 - accuracy: 1.0000
Epoch 8/10
154/154 [==============================] - 2s 12ms/step - loss: 6.4847e-04 - accuracy: 1.0000
Epoch 9/10
154/154 [==============================] - 1s 9ms/step - loss: 4.9976e-04 - accuracy: 1.0000
Epoch 10/10
154/154 [==============================] - 2s 10ms/step - loss: 3.988

In [6]:
# Evaluate Model to testing dataset
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

2/2 [==============================] - 2s 16ms/step - loss: 7.1522e-05 - accuracy: 1.0000
Test accuracy: 1.0
Test loss: 7.152248144848272e-05


In [7]:
# Try Predict to X_test data samples
# new_samples = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

new_samples = x_test[:5]
predictions = model.predict(new_samples)
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes = label_encoder.inverse_transform(predicted_classes)  # Mengembalikan label penyakit ke bentuk semula
print('Predicted classes:', predicted_classes)

1/1 [==============================] - 2s 2s/step
Predicted classes: ['Fungal infection' 'Allergy' 'GERD' 'Chronic cholestasis' 'Drug Reaction']


In [10]:
severityDictionary=dict()
description_list = dict()
precautionDictionary=dict()

symptoms_dict = {}

for index, symptom in enumerate(x_train):
  symptoms_dict[symptom] = index
def calc_condition(exp,days):
  sum=0
  for item in exp:
    sum=sum+severityDictionary[item]
  if((sum*days)/(len(exp)+1)>13):
    print("You should take the consultation from doctor. ")
  else:
    print("It might not be that bad but you should take precautions.")

In [11]:
def getDescription():
  global description_list
  with open('MasterData/symptom_Description.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
      _description={row[0]:row[1]}
      description_list.update(_description)

In [12]:
def getSeverityDict():
  global severityDictionary
  with open('MasterData/symptom_severity.csv') as csv_file:

    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    try:
      for row in csv_reader:
        _diction={row[0]:int(row[1])}
        severityDictionary.update(_diction)
    except:
      pass

In [13]:
def getprecautionDict():
  global precautionDictionary
  with open('MasterData/symptom_precaution.csv') as csv_file:

    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
      _prec={row[0]:[row[1],row[2],row[3],row[4]]}
      precautionDictionary.update(_prec)